In [ ]:
# Check whether 'beautifulsoup4' library is installed or not
!pip show beautifulsoup4

Name: beautifulsoup4
Version: 4.12.3
Summary: Screen-scraping library
Home-page: https://www.crummy.com/software/BeautifulSoup/bs4/
Author: 
Author-email: Leonard Richardson <leonardr@segfault.org>
License: MIT License
Location: /usr/local/lib/python3.10/dist-packages
Requires: soupsieve
Required-by: gdown, google, nbconvert, yfinance


In [ ]:
# Import packages
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [ ]:
# Import data
df = pd.read_excel("/content/drive/MyDrive/Web Scraping/BIN.xlsx")
df.head()

,bin
0,527351
1,540012
2,411911
3,535456
4,421808


In [ ]:
# Create get_bin() to crawl data from web
def get_bin(bin):
  search_url = f"https://bincheck.io/details/{bin}"
  response = requests.get(search_url)
  soup = BeautifulSoup(response.text, 'lxml')
  tables = soup.find_all('table', class_='w-full table-auto')   # Find tables
  data_dict = {}

  for table in tables:
      rows = table.find_all('tr')   # Find rows in table
      for row in rows:
          cells = row.find_all('td')    # Find cells in row
          if len(cells) > 1:  # Check if the row has at least 2 cells
              key = cells[0].text.strip()
              value = cells[1].text.strip()
              data_dict[key] = value  # Add key-value pair to the dictionary

  return data_dict  # Return the dictionary containing the data

In [ ]:
# Add new col using get_bin()
df['bin_data'] = df['bin'].apply(get_bin)

# Show data
print(df.head())

      bin                                           bin_data
0  527351  {'BIN/IIN': '527351', 'Card Brand': 'MASTERCAR...
1  540012  {'BIN/IIN': '540012', 'Card Brand': 'MASTERCAR...
2  411911  {'BIN/IIN': '411911', 'Card Brand': 'VISA', 'C...
3  535456  {'BIN/IIN': '535456', 'Card Brand': 'MASTERCAR...
4  421808  {'BIN/IIN': '421808', 'Card Brand': 'VISA', 'C...


In [ ]:
# Convert column 'bin_data' to new dataframe with keys = column headers and values = rows
bin_table = pd.json_normalize(df['bin_data'])

In [ ]:
# Show data
bin_table.head(10)

,BIN/IIN,Card Brand,Card Type,Card Level,Issuer Name / Bank,Issuer's / Bank's Website,Issuer / Bank Phone,ISO Country Name,Country Flag,ISO Country Code A2,ISO Country Code A3,ISO Country Currency
0,527351,MASTERCARD,DEBIT,PREPAID PLATINUM TRAVEL,"GPAY NETWORK (S) PTE., LTD.",------,+6566103888,SINGAPORE,,SG,SGP,SGD
1,540012,MASTERCARD,CREDIT,TITANIUM,"OVERSEA-CHINESE BANKING CORPORATION, LTD.",------,------,SINGAPORE,,SG,SGP,SGD
2,411911,VISA,CREDIT,SIGNATURE,"DBS BANK, LTD.",------,+6518001111111,SINGAPORE,,SG,SGP,SGD
3,535456,MASTERCARD,DEBIT,WORLD,"REVOLUT, LTD.",------,------,IRELAND,,IE,IRL,EUR
4,421808,VISA,DEBIT,PLATINUM,OVERSEA-CHINESE BANKING CORPORATION LTD.,------,------,SINGAPORE,,SG,SGP,SGD
5,552253,MASTERCARD,CREDIT,WORLD,"UNITED OVERSEAS BANK, LTD.",------,------,SINGAPORE,,SG,SGP,SGD
6,542550,MASTERCARD,CREDIT,WORLD,"CITIBANK SINGAPORE, LTD.",------,------,SINGAPORE,,SG,SGP,SGD
7,414746,VISA,CREDIT,SIGNATURE,CITIBANK SINGAPORE LTD.,------,------,SINGAPORE,,SG,SGP,SGD
8,426588,VISA,CREDIT,PLATINUM,"UNITED OVERSEAS BANK, LTD.",------,------,SINGAPORE,,SG,SGP,SGD
9,409636,VISA,DEBIT,BUSINESS ENHANCED,"DBS BANK, LTD.",------,+6518001111111,SINGAPORE,,SG,SGP,SGD


In [ ]:
# Remove unnecessary columns
bin_table.drop(columns={"Issuer's / Bank's Website",
                        "Issuer / Bank Phone",
                        "Country Flag",
                        "ISO Country Code A2",
                        "ISO Country Code A3"},
               axis=1, inplace=True)

In [ ]:
# Rename columns
bin_table.rename(columns={"Issuer Name / Bank": "Bank",
                          "ISO Country Name": "Country",
                          "ISO Country Currency": "Currency"},
                 inplace=True)

In [ ]:
# Add col 'Local/ Foreign'
bin_table['Local/ Foreign'] = np.where(bin_table['Country'] == 'SINGAPORE', 'Local', 'Foreign')

# Another way to create new col
# bin_table['Location'] = bin_table['Country'].apply(lambda x: 'Local' if x == 'SINGAPORE' else 'Foreign')

In [ ]:
bin_table.head()

,BIN/IIN,Card Brand,Card Type,Card Level,Bank,Country,Currency,Local/ Foreign
0,527351,MASTERCARD,DEBIT,PREPAID PLATINUM TRAVEL,"GPAY NETWORK (S) PTE., LTD.",SINGAPORE,SGD,Local
1,540012,MASTERCARD,CREDIT,TITANIUM,"OVERSEA-CHINESE BANKING CORPORATION, LTD.",SINGAPORE,SGD,Local
2,411911,VISA,CREDIT,SIGNATURE,"DBS BANK, LTD.",SINGAPORE,SGD,Local
3,535456,MASTERCARD,DEBIT,WORLD,"REVOLUT, LTD.",IRELAND,EUR,Foreign
4,421808,VISA,DEBIT,PLATINUM,OVERSEA-CHINESE BANKING CORPORATION LTD.,SINGAPORE,SGD,Local


In [ ]:
# Lưu lại dữ liệu vào file Excel
bin_table.to_excel('Bin_Data.xlsx', index=False)